# Objective

* Make a baseline model that predict the validation (28 days). 
* This competition has 2 stages, so the main objective is to make a model that can predict the demand for the next 28 days

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import dask_xgboost as xgb
import dask.dataframe as dd
from sklearn import preprocessing, metrics
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [102]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


# function to read the data and merge it (ignoring some columns, this is a very fst model)


def read_data():
    INPUT_DIR = '../data/input/'
    print('Reading files...')
    calendar = pd.read_csv(f'{INPUT_DIR}calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv(f'{INPUT_DIR}sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv(f'{INPUT_DIR}sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv(f'{INPUT_DIR}sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission


def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    # 
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    # get only a sample for fst training
    data = data.loc[nrows:]
    
    # drop some calendar features
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        # notebook crash with the entire dataset (maybee use tensorflow, dask, pyspark xD)
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        # get the sell price data (this feature should be very important)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data
        

In [103]:
%%time 
calendar, sell_prices, sales_train_validation, submission = read_data()
data = melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 27500000, merge = True)

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns
Melted sales train validation has 58327370 rows and 8 columns
Mem. usage decreased to 3226.27 Mb (9.4% reduction)
Our final dataset to train has 31681090 rows and 18 columns
CPU times: user 1min 31s, sys: 27.1 s, total: 1min 59s
Wall time: 2min


In [104]:
data.to_csv("preprocessed.csv", header=True)

* We have the data to build our first model, let's build a baseline and predict the validation data (in our case is test1)

In [105]:
tmp = pd.read_csv("preprocessed.csv", index=False)

In [106]:
tmp_reduced = reduce_mem_usage(tmp)

Mem. usage decreased to 3293.27 Mb (28.3% reduction)


In [107]:
tmp.head()

,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,0,HOUSEHOLD_1_514_WI_3_validation,HOUSEHOLD_1_514,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000
1,1,HOUSEHOLD_1_515_WI_3_validation,HOUSEHOLD_1_515,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703
2,2,HOUSEHOLD_1_516_WI_3_validation,HOUSEHOLD_1_516,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.980469
3,3,HOUSEHOLD_1_517_WI_3_validation,HOUSEHOLD_1_517,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.979980
4,4,HOUSEHOLD_1_518_WI_3_validation,HOUSEHOLD_1_518,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN


In [95]:
def labeling(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data

def add_feature(data):
    
    # rolling demand features
    grouped_demand = data.groupby(['id'])['demand']

#     data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
#     data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
#     data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    
    
    data['lag_t28'] = grouped_demand.transform(lambda x: x.shift(28))
    data['lag_t29'] = grouped_demand.transform(lambda x: x.shift(29))
    data['lag_t30'] = grouped_demand.transform(lambda x: x.shift(30))
#     data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
#     data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
#     data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
#     data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
#     data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
#     data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
#     data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
#     data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    
    
    # price features
#     data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
#     data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
#     data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
#     data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
#     data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
#     data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
#     data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    # time features
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    
    return data

def run_lgb(data):
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-03-27']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
    y_val = x_val['demand']
    test = data[(data['date'] > '2016-04-24')]
    del data
    gc.collect()

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 236,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}

    train_set = lgb.Dataset(x_train[features], y_train)
    val_set = lgb.Dataset(x_val[features], y_val)
    
    del x_train, y_train

    model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, valid_sets = [train_set, val_set], verbose_eval = 100)
    val_pred = model.predict(x_val[features])
    val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
    print(f'Our val rmse score is {val_score}')
    y_pred = model.predict(test[features])
    test['demand'] = y_pred
    return test

def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)
    

# define list of features
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30']


def feature_engineering(data):
    labeled_data = labeling(data)
    feature_added_data = add_feature(labeled_data)
    return feature_added_data

# def transform_train_and_eval(data):
#     data = transform(data)
#     data = simple_fe(data)
#     # reduce memory for new features so we can train
#     data = reduce_mem_usage(data)
#     test = run_lgb(data)
#     predict(test, submission)
    
# transform_train_and_eval(data)

In [90]:
small_data['lag_t28'] =  data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
# data['lag_t29'] = grouped_demand.transform(lambda x: x.shift(29))
# data['lag_t30'] = grouped_demand.transform(lambda x: x.shift(30))

In [92]:
small_data["lag_t28"].rolling(7).mean() 

30796880         NaN
30796881         NaN
30796882         NaN
30796883         NaN
30796884         NaN
              ...   
31681085    1.000000
31681086    0.857143
31681087    0.714286
31681088    0.857143
31681089    0.714286
Name: lag_t28, Length: 884210, dtype: float64

In [94]:
small_data["rolling_mean_t7"]

30796880    1.142857
30796881    0.571429
30796882    0.285714
30796883    3.571429
30796884    0.857143
              ...   
31681085    0.285714
31681086    0.142857
31681087    0.571429
31681088    1.142857
31681089    0.000000
Name: rolling_mean_t7, Length: 884210, dtype: float64

In [ ]:

data['lag_t28'] = grouped_demand.transform(lambda x: x.shift(28))
data['lag_t29'] = grouped_demand.transform(lambda x: x.shift(29))
data['lag_t30'] = grouped_demand.transform(lambda x: x.shift(30))
data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,16,train,2013-07-18,11325,30,4,2,2,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,1,train,2013-07-18,11325,30,4,2,2,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,1,train,2013-07-18,11325,30,4,2,2,0,0,0,1.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,1,train,2013-07-18,11325,30,4,2,2,0,0,0,0.979980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,0,train,2013-07-18,11325,30,4,2,2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31681085,FOODS_3_823_WI_3_validation,1432,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.980469,1.0,0.0,0.0,0.285714,0.487950,0.200000,0.433333
31681086,FOODS_3_824_WI_3_validation,1433,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,2.480469,0.0,1.0,0.0,0.142857,0.377964,0.300000,0.100000
31681087,FOODS_3_825_WI_3_validation,1434,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,3.980469,0.0,1.0,0.0,0.571429,0.786796,0.866667,0.900000
31681088,FOODS_3_826_WI_3_validation,1435,2,0,9,2,0,test1,2016-05-22,11617,30,4,2,2,0,0,0,1.280273,3.0,1.0,3.0,1.142857,1.345185,1.066667,1.022222


In [96]:
%%time 
small_data = data[data['date'] >= '2016-04-24']
fe_data = feature_engineering(small_data)

CPU times: user 25.8 s, sys: 208 ms, total: 26 s
Wall time: 26 s


In [98]:
fe_data.sort_values("id")

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_t28,lag_t29,lag_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,year,month,week,day,dayofweek
31133882,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,test1,2016-05-05,11614,0,0,0,0,1,1,1,2.240234,NaN,NaN,NaN,1.714286,1.889822e+00,1.200000,0.788889,2016,5,18,5,3
31042412,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,test1,2016-05-02,11614,4,2,0,0,1,0,1,2.240234,NaN,NaN,NaN,1.428571,1.812654e+00,1.066667,0.777778,2016,5,18,2,0
31194862,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,test1,2016-05-07,11615,4,2,0,0,1,1,0,2.240234,NaN,NaN,NaN,1.285714,1.603567e+00,1.200000,0.800000,2016,5,18,7,5
31408292,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,test1,2016-05-14,11616,4,2,0,0,0,0,1,2.240234,NaN,NaN,NaN,1.142857,8.997354e-01,1.200000,0.855556,2016,5,19,14,5
31072902,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,test1,2016-05-03,11614,4,2,0,0,1,1,1,2.240234,NaN,NaN,NaN,1.428571,1.812654e+00,1.100000,0.788889,2016,5,18,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31496712,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,0,test1,2016-05-16,11616,4,2,0,0,0,0,0,5.941406,NaN,NaN,NaN,0.000000,4.301595e-09,0.000000,0.044444,2016,5,20,16,0
31649162,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,0,test1,2016-05-21,11617,4,2,0,0,0,0,0,5.941406,NaN,NaN,NaN,0.000000,4.301595e-09,0.000000,0.044444,2016,5,20,21,5
30947892,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,0,test1,2016-04-28,11613,4,2,0,0,0,0,0,5.941406,NaN,NaN,NaN,0.000000,4.301595e-09,0.066667,0.077778,2016,4,17,28,3
31283282,HOUSEHOLD_2_516_WI_3_validation,3048,6,2,9,2,0,test1,2016-05-09,11615,4,2,0,0,1,1,1,5.941406,NaN,NaN,NaN,0.000000,4.301595e-09,0.033333,0.066667,2016,5,19,9,0
